In [ ]:
import os
import xarray as xr
import numpy as np
import glob
import pandas as pd
import numpy as np
import pickle
import math
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

maxuwind=-9999999  #This step is done so that later in the code you can normalize the data for easy comparison of variables.          
minuwind=999999    #You want to set the max to a really low number and the min to a really high number so that when you loop through the data you can essentially "beat" those values.

maxvwind=-9999999 #You will want to establish a max/min varible for each variable in your dataset that you are pulling (ex. If you want temperature make mintemp and maxtemp)
minvwind=999999

maxmslp=-9999999
minmslp=999999


mslpdata=np.empty([0,5916])  # Here I am making the new array that my flattened arrays will be placed. I had 3D arrays that needed to be flattened. 
uwinddata=np.empty([0,5916])
vwinddata=np.empty([0,5916])
alldata = np.empty([0,5916])

months=['01']         #Here I am selected the month of data I want. All data files are named MMYYYY.nc. You cna add multiple months here like so: ['01','02']
for m in months:

    files = glob.glob('DATA_ERA5\\'+m+'*',  #All data files in the form MMYYYY are in the folder 'DATA_ERA5'. This is simply the place your data is stored.
                   recursive = True)

    for f in files:
        ds= xr.open_dataset(f, engine='netcdf4')    #All the files I am using are xarrays and NetCDFs 
        
        mslp = (ds['msl'].values)/100
        lon = ds['longitude'].coarsen(longitude=4).mean()   #I define and pull the variables here. I use the lat and lon later in the code for plotting purposes 
        lat = ds['latitude'].values
        uwind = (ds['u10'].values)*1.94
        vwind = (ds['v10'].values)*1.94  #Conversions I do here 
        
        mslpraw=ds['msl'].coarsen(longitude=4).mean().coarsen(latitude=1).mean()/100  #coarsen is used to reduce the amount of data points. Here I am coarsening the longitude only by a factor of 4.
        uwindraw=ds['u10'].coarsen(longitude=4).mean().coarsen(latitude=1).mean()*1.94 #you need to coarsen each variable (ex. temp, relative humidity. etc) the same. Coarsening must result in an even number  
        vwindraw=ds['v10'].coarsen(longitude=4).mean().coarsen(latitude=1).mean()*1.94 #By even I mean, if you have 272 longitude points you can not coarsen by 5 because the ending value is not a whole number
#The conversions that are done in the coarsen step are NEEDED!!!! DO NOT FORGET THEM
    
        #generate the empty array that is the numnber of days in the month and the number of degrees of longitude*latatiude
        nday =int((ds['time'].size)) # if you want to have just one hour of the day divide by 24, however, I want all the hours so no division here
        nx = int((ds['latitude'].size)/1)
        ny = int((ds['longitude'].size)/4) # whatever factor you coarsen your data by you will want to divide the same variable here by that factor.
        mslparr = np.empty((nday, nx*ny))
        uwindarr = np.empty((nday, nx*ny))  #this array will house the flattened data so you need one for each vairable! I am getting my 3D data to be 2D
        vwindarr = np.empty((nday, nx*ny))

        for i in range(nday):   #loop here is done for each day 
            
            mslparr[i,:]= mslpraw[i,:,:].stack(point=["latitude", "longitude"])  #the stack function is reducing the dimensions of the original array (3D to 2D)
            uwindarr[i,:]= uwindraw[i,:,:].stack(point=["latitude", "longitude"])
            vwindarr[i,:]= vwindraw[i,:,:].stack(point=["latitude", "longitude"])
            
        mslpdata=np.append(mslpdata,mslparr,axis=0)   #you will use append here to place the flattened data into the arrya you made earlier
        uwinddata=np.append(uwinddata,uwindarr,axis=0)
        vwinddata=np.append(vwinddata,vwindarr,axis=0)
for i in range(23064):
    mslpdata[i,:] =mslpdata[i,:]-np.mean(mslpdata[i,:])    #the 23604 is from the fact that I am pulling Jnauary data so 31 days for 31 years and all 24 hours (31*31*24 == number of rows)
print(mslpdata)
#^^^the above code is used to get the anomalies in the MSLP. You can do this for any variable you have just make sure you loop through all the rows 
for i in range(23064):
    minmslp=min(minmslp,np.min(mslpdata[i,:]))
    maxmslp=max(maxmslp,np.max(mslpdata[i,:]))
for i in range(23064):
    minuwind=min(minuwind,np.min(uwinddata[i,:]))   #this code snipbits are used to determine the max and min of each row for each vairable. Each row correlates to one hour of the day for all 31 years and 31 days and 24 hours
    maxuwind=max(maxuwind,np.max(uwinddata[i,:]))
for i in range(23064):
    minvwind=min(minvwind,np.min(vwinddata[i,:]))
    maxvwind=max(maxvwind,np.max(vwinddata[i,:]))
print('##########################################')
mslp_factor=100./(maxmslp-minmslp)     #you are generating the normalization factor here. If you use wind, make sure to use the largest normalization factor for both the U/V wind. Do not use differnet values.
uwind_factor=100./(maxuwind-minuwind)
vwind_factor=100./(maxvwind-minvwind)
print('##########################################')
mslpdata=mslpdata*mslp_factor   #multiple the normalization factor with the correlating variable 
uwinddata=uwinddata*uwind_factor  #here the uwind facotr was largest so both get multiplied by it 
vwinddata=vwinddata*uwind_factor
print('##########################################')
alldata = np.hstack((mslpdata, uwinddata, vwinddata))   #the all data array is the final array and the input for the SOM. You stack horrizontally (add to the end of the row) the data so each row is one hour and all the data associated with that hour.